In [ ]:
#install.packages(c("tidyverse", "reshape2", "survminer", "survival", "splines", "coxme", "multcomp"))
library(tidyverse) # loads dplyr, ggplot2, stringr etc
library(reshape2) # for melt and cast

# The following packages are for analyzing mortality data and producing Kaplan-Meier plots:
library(survminer)
library(survival)
library(splines)
library(coxme)
library(multcomp)

In [ ]:
data <- read.csv("1_insect_mortality.csv",
                 header = TRUE)

fit <- survfit(Surv(time, death) ~ treatment, data = data)
print(fit, show.rmean=TRUE)
plot(fit, conf.int=TRUE)
summary(fit)

In [ ]:
survdiff(Surv(time,death)~treatment, data=data)

comp<-pairwise_survdiff(Surv(time, death) ~ treatment,
                        data = data, p.adjust.method = "fdr") #log rank compares the difference until an event (death) occurs
comp

In [ ]:
#Model with randomization effect by experiment
set.seed(2)
s1 = survfit(Surv(time,death)~treatment, data=data)
print(s1, show.rmean=TRUE)
plot(s1, conf.int=TRUE)
summary(s1)

In [ ]:
#Statistics
#Model random effect
data$treatment <- as.factor(data$treatment)
model <- coxme(Surv(time, death) ~ treatment+(1|biological_replicate), data=data)
anova(model)

summary(glht(model, linfct=mcp(treatment="Tukey")), test = adjusted(type = "bonferroni"))

In [ ]:
ggsurvplot(s1, data = data, 
           main = "Survival Feeding (11 Days)",
           size = 1, ggtheme = theme_classic(), 
           linetype=c("solid", "dotted", "dotdash", "dashed"), 
           xlab="Days", ylab="% of survival", palette=c("#887bb9", "#a35da5", "#69b19a", "#8ec372"))

In [ ]:
# Midpoint and endpoint snapshot

#Calculate survival at endpoint
time_point = 11 # change for wanted timepoint
mortality_at_timepoint = data.frame(Soil = c("C10", "C112", "S16", "S7"), Mortality = (1 - summary(s1, times= time_point, extend = T)$surv)*100)


p <- ggplot(mortality_at_timepoint, aes(x = Soil, y = Mortality)) +
        geom_bar(position="stack", stat = "identity") + 
        theme_classic() 

p


#Mortality-benzoxazinoid correlation plots
#change either use the data on Benzoxanzinoids or Phytohormones.
benz = left_join(read.csv("4_benzoxazinoids.csv", header = T), mortality_at_timepoint, by="Soil")

ggplot(benz, aes(x=Concentration_mg_g, y=Mortality))+
geom_point(shape=16, aes(color=Soil))+
stat_cor(method = "spearman")+
geom_smooth(method = "lm", formula = y ~ x, se = T, color = "black") +
scale_y_continuous(limits = c(0, 100))+
ylab("Mortality (%)") + xlab("Compound concentration (ng/g)")+
scale_color_manual(values=c("#887bb9", "#a35da5", "#69b19a", "#8ec372"))+
facet_wrap(~Treatment+Benzoxazinoid+Sample_Type, scales = "free_x", ncol = 5)

#The plot needs to be saved in a large pdf to be able to fully visualize each facet.